In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch, wandb, platform, warnings
from datasets import load_dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login

# Define model

In [2]:
device = "cuda" # the device to load the model onto
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", token='TOKENS A AJOUTER') 

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [3]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2", 
    quantization_config=bnb_config,
    device_map={"": 0},
    token='TOKENS A AJOUTER''
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

# Fonction message 

In [4]:
def query (text):
    return [
    {"role": "user", "content": 'CONTINUE THE EXEMPLE BY CHOSING ONE OF THE FOLOWING WORDS : "Politics",  "Health",  "Finance", "Travel", "Food", "Education", "Environment", "Fashion",   "Science",  "Sports", "Technology",  "Entertainment"\
RESPOND WITH ONLY ONE WORD \
EXEMPLE : \
"The mayor announced a new initiative to improve public transportation." => "Politics"\
"The stock market saw a significant drop following the announcement." => "Finance"\
"' + text + '" => \
'},
]

# Test model

In [5]:
message = query ("The mayor announced a new initiative to improve public transportation.")
print(message)
encodeds = tokenizer.apply_chat_template(message, return_tensors="pt")
model_inputs = encodeds.to(device)
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
tokenizer.batch_decode(generated_ids)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'role': 'user', 'content': 'CONTINUE THE EXEMPLE BY CHOSING ONE OF THE FOLOWING WORDS : "Politics",  "Health",  "Finance", "Travel", "Food", "Education", "Environment", "Fashion",   "Science",  "Sports", "Technology",  "Entertainment"RESPOND WITH ONLY ONE WORD EXEMPLE : "The mayor announced a new initiative to improve public transportation." => "Politics""The stock market saw a significant drop following the announcement." => "Finance""The mayor announced a new initiative to improve public transportation." => '}]


['<s> [INST] CONTINUE THE EXEMPLE BY CHOSING ONE OF THE FOLOWING WORDS : "Politics",  "Health",  "Finance", "Travel", "Food", "Education", "Environment", "Fashion",   "Science",  "Sports", "Technology",  "Entertainment"RESPOND WITH ONLY ONE WORD EXEMPLE : "The mayor announced a new initiative to improve public transportation." => "Politics""The stock market saw a significant drop following the announcement." => "Finance""The mayor announced a new initiative to improve public transportation." =>  [/INST] Politics</s>']

# Fine tune model 

In [6]:
# meilleur data set d'apprentissage
Data = {
  "Politics": [
    "The mayor announced a new initiative to improve public transportation.",
    "The senator is facing criticism for her stance on the recent bill.",
    "The upcoming election has sparked intense debates among the candidates.",
    "A local government is launching a campaign to increase voter registration rates.",
    "New legislation is being considered to address climate change at a national level."
  ],
  "Health": [
    "Regular exercise and a balanced diet are key to maintaining good health.",
    "The World Health Organization has issued new guidelines on COVID-19.",
    "A new study reveals the benefits of meditation for mental health.",
    "Healthcare reform is being debated as a crucial issue for upcoming elections.",
    "Breakthroughs in cancer research promise new treatment options."
  ],
  "Finance": [
    "The stock market saw a significant drop following the announcement.",
    "Investing in real estate can be a profitable venture if done correctly.",
    "The company's profits have doubled since the launch of their new product.",
    "Cryptocurrency regulations are being discussed by governments worldwide.",
    "Financial technology innovations continue to disrupt traditional banking."
  ],
  "Travel": [
    "Visiting the Grand Canyon is a breathtaking experience.",
    "The tourism industry has been severely impacted by the pandemic.",
    "Backpacking through Europe is a popular choice for young travelers.",
    "Eco-tourism is gaining popularity as travelers seek more sustainable options.",
    "New travel apps are making international travel more accessible and organized."
  ],
  "Food": [
    "The new restaurant in town offers a fusion of Italian and Japanese cuisine.",
    "Cooking classes are a fun way to learn new recipes and techniques.",
    "Farm-to-table movements are influencing restaurants to adopt local sourcing.",
    "Food festivals are becoming major events in cities around the world.",
    "Nutritionists emphasize the importance of diverse diets to improve health."
  ],
  "Education": [
    "The school district is implementing a new curriculum for the upcoming year.",
    "Online learning has become increasingly popular during the pandemic.",
    "The university is offering scholarships for students in financial need.",
    "There is growing interest in bilingual education programs.",
    "Educational technology startups are developing new tools for classroom learning."
  ],
  "Environment": [
    "Climate change is causing a significant rise in sea levels.",
    "Recycling and composting are effective ways to reduce waste.",
    "The Amazon rainforest is home to millions of unique species.",
    "Urban green spaces are being expanded to improve air quality.",
    "Oceans are being protected through international agreements on marine conservation."
  ],
  "Fashion": [
    "The new fashion trend is all about sustainability and eco-friendly materials.",
    "The annual Met Gala is a major event in the fashion world.",
    "Vintage clothing has made a comeback in recent years.",
    "Technology in fashion is leading to smarter wearables.",
    "Fashion design schools are promoting ethical fashion as part of their curriculum."
  ],
  "Science": [
    "NASA's Mars Rover has made significant discoveries about the red planet.",
    "The Nobel Prize in Physics was awarded for breakthroughs in black hole research.",
    "Genetic engineering is opening up new possibilities in medical treatment.",
    "Quantum computing is seen as the next major frontier in technology.",
    "Sustainable energy sources are being developed at a faster rate due to technological advancements."
  ],
  "Sports": [
    "The NBA Finals are set to begin next week with the top two teams in the league.",
    "Serena Williams continues to dominate the tennis world with her powerful serve.",
    "The World Cup is the most prestigious tournament in international soccer.",
    "Extreme sports are gaining a larger audience through online streaming platforms.",
    "Youth sports programs are receiving increased funding to promote physical education."
  ],
  "Technology": [
    "Artificial intelligence is changing the way we live and work.",
    "The latest iPhone has a number of exciting new features.",
    "Cybersecurity is becoming increasingly important as more and more data moves online.",
    "The rise of remote work has led to the development of new virtual collaboration tools.",
    "Blockchain technology is being adopted for more than just cryptocurrencies."
  ],
  "Entertainment": [
    "The new Marvel movie is breaking box office records.",
    "The Grammy Awards are a celebration of the best music of the year.",
    "The latest season of Game of Thrones had fans on the edge of their seats.",
    "The benefits of using innovative materials in fashion are many, including improved performance and reduced environmental impact.",
    "Streaming services are investing heavily in original content to capture global audiences."
  ]
}


In [7]:
from transformers import AutoTokenizer
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
import pandas as pd
import torch

tokenizer.pad_token = tokenizer.eos_token

model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

# Generate training set
train_set = []
for category, sentences in Data.items():
    for sentence in sentences:
        processed_sentence = query(sentence)[0]['content']
        message = f'<s> [INST] {processed_sentence} [/INST] {category} </s>'
        encoded_message = tokenizer(message, truncation=True, max_length=512, add_special_tokens=True)
        train_set.append({'input_ids': encoded_message['input_ids'], 'labels': encoded_message['input_ids']}) # Labels are the same for LM

# Convert list to DataFrame and then to Hugging Face Dataset
df = pd.DataFrame(train_set)
dataset = Dataset.from_pandas(df)
dataset = dataset.map(lambda examples: {'input_ids': examples['input_ids'], 'labels': examples['labels']}, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    num_train_epochs=8,
    weight_decay=0.01,
    save_strategy="no",
    logging_dir='./logs',
    logging_steps=100,
    report_to="none"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

# Start training
trainer.train()

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/theotime/miniconda3/envs/AI/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
/home/theotime/miniconda3/envs/AI/lib/python3.8/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will

Step,Training Loss
100,0.661700
200,0.204600
300,0.131300
400,0.067800


TrainOutput(global_step=480, training_loss=0.22997719347476958, metrics={'train_runtime': 133.0267, 'train_samples_per_second': 3.608, 'train_steps_per_second': 3.608, 'total_flos': 3165663478087680.0, 'train_loss': 0.22997719347476958, 'epoch': 8.0})

# Soirtie model

In [8]:
# Open the file
with open('data/test_shuffle.txt', 'r') as file:
    # Read lines from the file
    lines = file.read().split('\n')

# lines now contains each line of the file as an element in the list
#print(lines)

In [9]:
with open('output.txt', 'w') as outfile:
    for text in lines : 
        # get query 
        message = query (text)

        # Encode messages using the tokenizer
        encodeds = tokenizer.apply_chat_template(message, return_tensors="pt")

        # Move encoded inputs to the GPU and ensure they are in half precision
        model_inputs = encodeds.to(device)

        # Generate responses without calculating gradients
        with torch.no_grad():
            generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)

        # Decode generated ids back to text
        decoded = tokenizer.batch_decode(generated_ids)

        # TODO write decoded[0] which is a text in a output file
        outfile.write(decoded[0] + "\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/theotime/miniconda3/envs/AI/lib/python3.8/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for 

In [11]:
assert 1 == 2

AssertionError: 

# Read, check and transform output for competition 

In [16]:
import csv
import re
import random

# Define the file path
file_path = 'output.txt'  # Replace 'path_to_your_file.txt' with your actual file path

# Function to read the file
def read_file_to_text(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

# Read the file and store the content in a variable
input_text = read_file_to_text(file_path)


# Categories
valid_categories = ["Politics", "Health", "Finance", "Travel", "Food", "Education", "Environment", "Fashion", "Science", "Sports", "Technology", "Entertainment"]


def parse_responses_with_full_line_warning(input_text):
    results = []
    snippets = input_text.split("<s>")
    # List of words to ignore as they are not categories
    ignore_list = ['INST', 's']
    for snippet in snippets:
        if "=>" in snippet:
            # Extract the last part after '=>'
            response_part = snippet.split("=>")[-1]
            # Use regex to extract only valid category words by ignoring all non-alphabetic characters before and after the category
            matches = re.findall(r'(\b\w+\b)', response_part)
            for match in matches:
                if match in valid_categories:
                    results.append(match)
                elif match not in ignore_list:
                    print(f"Warning: '{match}' is not a valid category or unnecessary text. Full line: {snippet.strip()}")
    return results

# Validate and generate CSV
def generate_csv(responses):
    with open('categories.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ID", "Label"])
        for idx, response in enumerate(responses):
            if response in valid_categories:
                writer.writerow([idx, response])
            else:
                print(f"Warning: '{response}' is not a valid category.")
    print("CSV file has been generated.")

# Parse input text


responses = parse_responses_with_full_line_warning(input_text)


generate_csv(responses)

CSV file has been generated.
